#### Trabajo Práctico N° 2
##### Sistemas de Recomendación
Facultad de Ciencias Exactas y Naturales

Álgebra Lineal Computacional 2°C 2023

Luca Jaichenco ~ 591/22 |Mario Sigal Aguirre ~ 157/22

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Ejercicio 1: Pasos para implementar un sistema de recomendación de vinos

Se siguieron los pasos descritos en el trabajo practico y en el archivo auxiliar para la creación de un sistema de recomendación de vinos.

#### (a) Descargar los datos

In [2]:
df_wine = pd.read_csv('./wine.csv')

#### (b) Separar los datos en variables independientes (X) y dependiente (Y).

In [3]:
# Variables independientes:
variables_independientes = ['Alcohol', 'Malic_Acid', 'Ash', 'Ash_Alcanity', 'Magnesium',
                            'Total_Phenols', 'Flavanoids', 'Nonflavanoid_Phenols',
                            'Proanthocyanins', 'Color_Intensity', 'Hue', 'OD280', 'Proline']
df_wine_independientes = df_wine[variables_independientes]

# Variable dependiente:
variables_dependientes =['Customer_Segment']
df_wine_dependientes = df_wine[variables_dependientes]

#### (c) Normalizar y centrar los datos.
Se pide construir una función en python que normalice y centre los valores respecto del promedio. Es decir, aplicar a cada variable independiente:
1. $ xi = (X_i - \bar{X})/s $ 

2. $ \bar{X} = (\sum_{i=1}^{N} X_i) /N  $

3. $ s = \sqrt{\frac{1}{N-1} \sum_{i=1}^{N} (X_i - \bar{X})^2} $

In [4]:
#función que normaliza y centra respecto del promedio 
def normalizacion(df):# recibe un Data Frame que contenga solamente las variables independientes    
    
    # inicialización de variables
    df_norm = pd.DataFrame() 
    
    # Normalización
    for i in df.columns:
        # normalización de cada atributo
        X= df[i] 
        X_prom = X.mean() # calculamos el promedio     # usar pandas o programarlas?
        s = X.std() #calculamos la desviación estándar
        X_norm = [] 
        
        for j in range(len(X)): 
            # normalización de cada elemento
            xj= (X[j] - X_prom)/s
            X_norm.append(xj) 
        
        df_norm[i] = X_norm #agrega cada atributo normalizado al data frame normalizado
    
    return df_norm

df_wine_independientes_norm = normalizacion(df_wine_independientes)

Normalizar un conjunto de datos es importante por varias razones:
En principio queremos que las variables tengan las mismas escala y mismo rango, pues de no tenerlas, la matriz de covarianza podría verse afectada, dando más peso a las variables con escalas más grandes y haciendo que las variables con rangos más pequeños tengan una contribución mínima. La normalización asegura que todas las variables tengan la misma escala y rango.
A su ves  la normalización facilita la interpretación de la matriz de covarianza y de los resultados de PCA ya que como los componentes principales proporcionan información sobre la contribución de cada variable a la variabilidad total, normalizar hace que nos sea posible comparar estas contribuciones de manera optima.

#### (d) Calcular la matriz de covarianza
$cov(b) = \frac{b^{*}b}{n-1}$ 

In [5]:
#pasamos el data frame a una matriz
matriz_wine_norm = df_wine_independientes_norm.values

#calculamos la matriz de covarianza
wine_cov = (matriz_wine_norm.T @ matriz_wine_norm)/177

La matriz de covarianza describe la relación entre las variables de nuestro data frame. Los elementos en la posición $(i,j)$  de la matriz se encuentran entre menos uno y uno [-1,1] y representan la covarianza entre las variables $ X_i$ y $X_j$: 
Si el elemento es cercano a uno indica una relación positiva, Cuando $X_i$ tiende a ser grande, $X_j$ también tiende a ser grande, y viceversa.
Si el elemento es cercano a menos uno indica una relación negativa. Cuando $X_i$ tiende a ser grande,$X_j$ tiende a ser pequeño, y viceversa.
Si el elemento es cercano a cero indica que no existe una relación precisa 
Es de dimension $n \times n$ y en nuestro caso $ 13 \times 13. cumple las propiedades de ser Simétrica y Definida Positiva

#### (e) Encontrar el máximo autovalor y el correspondiente autovector de la matriz de covarianza  por el método de la potencia

In [6]:

def metodo_potencia(m, its=100):
  
    v = np.random.rand(m.shape[0])  # Inicializar un vector aleatorio
    for i in range(its):
        v = np.dot(m, v)
        v = v / np.linalg.norm(v)  # Normalizar el vector en cada iteración
    
    AutoValor = np.dot(np.dot(m, v), v) / np.dot(v, v) 
    AutoVector = v
    return AutoValor, AutoVector

max_Aval_wine_cov, Avec_de_max_Aval_wine_cov = metodo_potencia(wine_cov)

print(max_Aval_wine_cov, Avec_de_max_Aval_wine_cov)

4.705850252990422 [ 0.1443294  -0.24518758 -0.00205106 -0.23932041  0.14199204  0.39466085
  0.4229343  -0.2985331   0.31342949 -0.0886167   0.29671456  0.37616741
  0.28675223]


In [7]:
#(f)
def todos_Avals_Avects(A, n):
    Avals = []
    Avects = []
    Aval = 0
    Avect = np.zeros(A.shape[0])
    for i in range(0,n):
        A = A - Aval*np.outer(Avect,Avect)
        Aval, Avect = metodo_potencia(A)
        Avals.append(round(Aval, 5))
        Avects.append(Avect)
    return Avals, Avects

A= np.array([[6,-1],[2,3]])   
Avals, V = todos_Avals_Avects(A,2)
    
print(Avals, V)


[5.0, 4.0] [array([0.70710678, 0.70710678]), array([ 0.98994949, -0.14142136])]


In [8]:
#(g)
# buscamos autovalores y autovectores de la matriz de covarianza
Avals, Avects = todos_Avals_Avects(wine_cov, 13)

# Sea V  la matriz con los autovectores por columnas  
V = np.array(Avects).T
# Sea D la matriz diagonal con los autovalores en la diagonal ordenados de mayor a menor
D = np.diag(Avals)

# decidimos tomar como los autovalores mas relevantes a los primeros 3

In [9]:
def distancia(v1,v2):
    x = 0
    for i in  range(v1.shape[0]):
        x += (v1[i]-v2[i])**2
    return np.sqrt(x)


def KNN(T_test, T_train, Y_train, K): # devuelve Y_estimado
    Y_estimado = []
    for i in range(len(T_test)):
        # Calculo las distancias
        distancias = []
        for j in range(len(T_train)):
            distancias.append((distancia(T_test[i],T_train[j]), j))
        # Ordeno las distancias, manteniendo el indice de la fila
        distancias.sort()
        # Calculo los resultados
        resultados = {
            1:0,
            2:0,
            3:0
        }
        for k in range(K):
            resultados[Y_train[distancias[k][1]]] += 1
        # Agrego el resultado que mas veces apareció en los k vecinos mas cercanos
        Y_estimado.append(max(resultados, key=resultados.get))
    return Y_estimado

In [10]:
# Separo los datos en train y test
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Dividimos el dataframe normalizado en casos de train y en casos de test
X_train, X_test, Y_train, Y_test = train_test_split(df_wine_norm, df_wine_dependientes, test_size=0.2, random_state=0)

# Definimos la matriz de covarianza para los casos de train
X_train_cov = (X_train.T @ X_train)/len(X_train)

# buscamos autovalores y autovectores de la matriz de covarianza
Avals, Avects = todos_Avals_Avects(X_train_cov, 13)

# Sea V  la matriz con los autovectores por columnas  
V = np.array(Avects).T

CP = 4 # Cantidad de componentes principales
K = 5 # Nearest neigbours

# Calculo las componentes principales
T_train = (X_train @ V[:,:CP]).values # el .values convierte el df en un np array
T_test = (X_test @ V[:,:CP]).values

Y_train = Y_train['Customer_Segment'].to_numpy() # Agarra la columna 'Customer_Segment' y la convierte en un np array
Y_test = Y_test['Customer_Segment'].to_numpy()

# Calculo el Y estimado
Y_estimado = KNN(T_test, T_train, Y_train, K)

# Calculo el error
error = 0
for i in range(len(Y_test)):
    if Y_test[i] != Y_estimado[i]:
        error += 1
error = error/len(Y_test)

print(error)

0.05555555555555555
